In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rohanrao/air-quality-data-in-india")

In [ ]:
import pandas as pd
path1="assignment/dataset/city_day.csv"
path2="/assignment/dataset/city_hour.csv"
data_daily = pd.read_csv(path1)
data_hourly=pd.read_csv(path2)
data_daily.head()

In [ ]:
data_daily=data_daily.dropna(subset=['AQI'])
data_hourly=data_hourly.dropna(subset=['AQI'])
#dropping every column except AQI, date and City
data_daily=data_daily[['City','Date','AQI']]
data_hourly=data_hourly[['City','Datetime','AQI']]
data_daily.info()

In [ ]:
cities=['Delhi','Kolkata']
data_daily=data_daily[data_daily['City'].isin(cities)]
data_daily['Date']=pd.to_datetime(data_daily['Date'])
data_hourly=data_hourly[data_hourly['City'].isin(cities)]
data_hourly['DateT']=pd.to_datetime(data_hourly['Datetime'])
data_daily.describe()


In [ ]:
delhi_daily=data_daily[data_daily['City']=='Delhi']
kolkata_daily=data_daily[data_daily['City']=='Kolkata']

delhi_hourly=data_hourly[data_hourly['City']=='Delhi']
kolkata_hourly=data_hourly[data_hourly['City']=='Kolkata']


In [ ]:
print(delhi_daily['Date'].min())
print(delhi_daily['Date'].max())
print(kolkata_daily['Date'].min())
print(kolkata_daily['Date'].max())
# removing dates before 2018-05-07 00:00:00 from city1
delhi_daily=delhi_daily[delhi_daily['Date']>='2018-07-01 00:00:00']
kolkata_daily=kolkata_daily[kolkata_daily['Date']>='2018-07-01 00:00:00']

In [ ]:
print(delhi_hourly['DateT'].min())
print(delhi_hourly['DateT'].max())
print(kolkata_hourly['DateT'].min())
print(kolkata_hourly['DateT'].max())
# removing dates before 2018-05-07 00:00:00 from city1
delhi_hourly=delhi_hourly[delhi_hourly['DateT']>='2018-07-01 00:00:00']
kolkata_hourly=kolkata_hourly[kolkata_hourly['DateT']>='2018-07-01 00:00:00']

In [ ]:
# Plotting the data
import matplotlib.pyplot as plt
plt.figure(figsize=(15,6))
plt.plot(delhi_daily['Date'],delhi_daily['AQI'],label='Delhi')
plt.plot(kolkata_daily['Date'],kolkata_daily['AQI'],label='Kolkata')
plt.xlabel('Date-Time')
plt.ylabel('AQI')
plt.legend()
plt.show()


In [ ]:
# Plotting the data
import matplotlib.pyplot as plt
plt.figure(figsize=(15,6))
plt.plot(delhi_hourly['DateT'],delhi_hourly['AQI'],label='Delhi')
plt.plot(kolkata_hourly['DateT'],kolkata_hourly['AQI'],label='Kolkata')
plt.xlabel('Date-Time')
plt.ylabel('AQI')
plt.legend()
plt.show()


In [ ]:
def calculate_monthly_stats(city_data):
    monthly_stats = {}
    for year in city_data['Date'].dt.year.unique():
        monthly_stats[year] = {}
        for month in range(1, 13):
            # Filter data for the specific year and month
            month_data = city_data[(city_data['Date'].dt.year == year) & (city_data['Date'].dt.month == month)]
            
            if not month_data.empty:
                # Calculate mean and variance for each city within this month
                monthly_stats[year][month] = month_data.groupby('City')['AQI'].agg(['mean', 'var']).to_dict(orient='index')
                
    # Convert to DataFrame for easier viewing
    monthly_stats_df = pd.DataFrame.from_dict({(year, month): monthly_stats[year][month] 
                                               for year in monthly_stats.keys() 
                                               for month in monthly_stats[year].keys()},
                                              orient='index')
    return monthly_stats_df

monthly_stats_delhi = calculate_monthly_stats(delhi_daily)
monthly_stats_kolkata = calculate_monthly_stats(kolkata_daily)

In [ ]:
# Plotting monthly mean AQI for Delhi and Mumbai
plt.figure(figsize=(15,6))
monthly_stats_delhi['Delhi'].apply(lambda x: x['mean']).plot(label='Delhi')
monthly_stats_kolkata['Kolkata'].apply(lambda x: x['mean']).plot(label='Kolkata')
plt.xlabel('Year-Month')
plt.ylabel('Mean AQI')
plt.legend()
plt.xticks(ticks=range(len(monthly_stats_delhi.index)), labels=[f"{year}-{month:02d}" for year, month in monthly_stats_delhi.index], rotation=45)
plt.show()


In [ ]:
# Plotting monthly variance of AQI for Delhi and Mumbai
plt.figure(figsize=(15,6))
monthly_stats_delhi['Delhi'].apply(lambda x: x['var']).plot(label='Delhi')
monthly_stats_kolkata['Kolkata'].apply(lambda x: x['var']).plot(label='Kolkata')
plt.xlabel('Year-Month')
plt.ylabel('Variance AQI')
plt.legend()
plt.xticks(ticks=range(len(monthly_stats_delhi.index)), labels=[f"{year}-{month:02d}" for year, month in monthly_stats_delhi.index], rotation=45)
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
axes[0].hist(delhi_daily['AQI'], bins=20,color='green' ,edgecolor='black')
axes[0].set_title('Delhi AQI')
axes[0].set_xlabel('AQI Value')
axes[0].set_ylabel('Number of Days')

# Kolkata AQI Histogram
axes[1].hist(kolkata_daily['AQI'], bins=20, edgecolor='black')
axes[1].set_title('Kolkata')
axes[1].set_xlabel('AQI Value')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
axes[0].hist(delhi_hourly['AQI'], bins=20,color='green' ,edgecolor='black')
axes[0].set_title('Delhi AQI')
axes[0].set_xlabel('AQI Value')
axes[0].set_ylabel('Number of Hours')

# Kolkata AQI Histogram
axes[1].hist(kolkata_hourly['AQI'], bins=20, edgecolor='black')
axes[1].set_title('Kolkata')
axes[1].set_xlabel('AQI Value')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
x=delhi_daily[(delhi_daily['City'] == 'Delhi') & (delhi_daily['Date'] >= '2018-07-01') & (delhi_daily['Date'] <= '2020-07-01')]
aqi_counts = x['AQI'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
plt.scatter(aqi_counts.index, aqi_counts.values, color='blue', edgecolor='black')
plt.xlabel('AQI Value')
plt.ylabel('Number of Days')
plt.show()

In [ ]:
x=delhi_hourly[(delhi_hourly['City'] == 'Delhi') & (delhi_hourly['DateT'] >= '2018-07-01') & (delhi_hourly['DateT'] <= '2020-07-01')]
aqi_counts = x['AQI'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
plt.scatter(aqi_counts.index, aqi_counts.values, color='blue', edgecolor='black')
plt.xlabel('AQI Value')
plt.ylabel('Number of Hours')
plt.show()

In [ ]:
delhi_daily=delhi_daily[delhi_daily['Date']<='2020-05-01 00:00:00']
kolkata_daily=kolkata_daily[kolkata_daily['Date']<='2020-05-01 00:00:00']
delhi_hourly=delhi_hourly[delhi_hourly['DateT']<='2020-05-01 00:00:00']
kolkata_hourly=kolkata_hourly[kolkata_hourly['DateT']<='2020-05-01 00:00:00']

In [ ]:

def calculate_aqi_stats(city_data, start_date, end_date):
    # Filter data for the specified date range
    filtered_data = city_data[(city_data['DateT'] >= start_date) & (city_data['DateT'] <= end_date)]
    
    # Calculate mean and variance
    mean_aqi = filtered_data['AQI'].mean()
    variance_aqi = filtered_data['AQI'].var()
    
    # Store the results in a dictionary
    aqi_stats = {
        'mean': mean_aqi,
        'variance': variance_aqi
    }
    
    return aqi_stats

# Example usage:
start_date = '2018-07-01'
end_date = '2020-07-01'
delhi_aqi_stats = calculate_aqi_stats(delhi_hourly, start_date, end_date)
kolkata_aqi_stats = calculate_aqi_stats(kolkata_hourly, start_date, end_date)

print("Delhi AQI Stats:", delhi_aqi_stats)
print("Kolkata AQI Stats:", kolkata_aqi_stats)


In [ ]:
from pandas.tseries.offsets import DateOffset

def calculate_aqi_stats_intervals(city_data, start_date, end_date, interval_months=2):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    current_start = start_date
    means = []
    variances = []
    
    while current_start < end_date:
        current_end = current_start + DateOffset(months=interval_months)
        if current_end > end_date:
            current_end = end_date
        
        stats = calculate_aqi_stats(city_data, current_start, current_end)
        means.append(stats['mean'])
        variances.append(stats['variance'])
        
        current_start = current_end
    
    return means, variances

# Example usage:
start_date = '2018-07-01'
end_date = '2020-07-01'
delhi_aqi_stats_intervals = calculate_aqi_stats_intervals(delhi_hourly, start_date, end_date)
kolkata_aqi_stats_intervals = calculate_aqi_stats_intervals(kolkata_hourly, start_date, end_date)

print("Delhi AQI Stats Intervals:", delhi_aqi_stats_intervals)
print("Kolkata AQI Stats Intervals:", kolkata_aqi_stats_intervals)
len(delhi_aqi_stats_intervals[1])

In [ ]:
import numpy as np

# Function to remove NaN values from lists
def remove_nan_values(stats_intervals):
    means, variances = stats_intervals
    means = [value for value in means if not np.isnan(value)]
    variances = [value for value in variances if not np.isnan(value)]
    return means, variances

# Remove NaN values from the intervals
delhi_aqi_stats_intervals = remove_nan_values(delhi_aqi_stats_intervals)
kolkata_aqi_stats_intervals = remove_nan_values(kolkata_aqi_stats_intervals)

print("Delhi AQI Stats Intervals without NaN:", delhi_aqi_stats_intervals)
print("Kolkata AQI Stats Intervals without NaN:", kolkata_aqi_stats_intervals)
# Adding another value to the intervals
delhi_aqi_stats_intervals[1].append(1444.38)

kolkata_aqi_stats_intervals[1].append(1243.83)

print("Updated Delhi AQI Stats Intervals:", delhi_aqi_stats_intervals)
print("Updated Kolkata AQI Stats Intervals:", kolkata_aqi_stats_intervals)

In [ ]:
#calculating mean and variance of Delhi AQI Stats Intervals
import numpy as np
def calculate_mean_variance(aqi_intervals):
    means, variances = aqi_intervals
    sample_mean = np.mean(means)
    sample_variance = np.var(variances, ddof=1)
    return sample_mean, sample_variance

In [ ]:
mean_delhi, variance_delhi = calculate_mean_variance(delhi_aqi_stats_intervals)
mean_delhi, variance_delhi


In [ ]:

mean_kolkata, variance_kolkata = calculate_mean_variance(kolkata_aqi_stats_intervals)
mean_kolkata, variance_kolkata


In [ ]:
def standard_error_of_difference(var1, var2, n1, n2):
    """Calculate the standard error of the difference between two means."""
    se = np.sqrt((var1 / n1) + (var2 / n2))
    return se

In [ ]:
from scipy.stats import norm
mean_dff=mean_delhi-mean_kolkata
sdd= standard_error_of_difference(variance_delhi, variance_kolkata, 360, 360)
mean_dff, sdd


In [ ]:

z_critical = norm.ppf(0.975) 
ci_lower = mean_dff - z_critical * sdd  
ci_upper = mean_dff + z_critical * sdd
ci_lower, ci_upper